## Install Crew

In [1]:
!uv add crewai 'crewai-tools>0.40.0' 'langchain-openai>=0.2.1' embedchain


    █████▒██▀███  ▓█████ ▄▄▄       ██ ▄█▀     Hostname: Freak-PC 
  ▓██   ▒▓██ ▒ ██▒▓█   ▀▒████▄     ██▄█▒     
  ▒████ ░▓██ ░▄█ ▒▒███  ▒██  ▀█▄  ▓███▄░      OS: EndeavourOS 
  ░▓█▒  ░▒██▀▀█▄  ▒▓█  ▄░██▄▄▄▄██ ▓██ █▄     
  ░▒█░   ░██▓ ▒██▒░▒████▒▓█   ▓██▒▒██▒ █▄     User: Freak 
  ▒ ░   ░ ▒▓ ░▒▓░░░ ▒░ ░▒▒   ▓▒█░▒ ▒▒ ▓▒     
   ░       ░▒ ░ ▒░ ░ ░  ░ ▒   ▒▒ ░░ ░▒ ▒░     Date: 11 June 2025 
   ░ ░     ░░   ░    ░    ░   ▒   ░ ░░ ░     
            ░        ░  ░     ░  ░░  ░       

Resolved 305 packages in 1ms
Audited 293 packages in 0.21ms


### Import Required

In [2]:
from crewai import Agent, Crew, Process, Task, LLM

### Create Agents

In [3]:
engineering_lead = Agent(
        role="Engineering Lead for the engineering team, directing the work of the engineer",
        goal="""Take the high level requirements described here and prepare a detailed design for the backend developer;
            everything should be in 1 python module; describe the function and method signatures in the module.
            The python module must be completely self-contained, and ready so that it can be tested or have a simple UI built for it.
            Here are the requirements: {requirements}
            The module should be named {module_name} and the class should be named {class_name}""",
        backstory="""You're a seasoned engineering lead with a knack for writing clear and concise designs.""",
        verbose=True,
        llm=LLM(model="ollama/llama3.2:1b", base_url="http://localhost:11434"),
    )

backend_engineer = Agent(
        role="Python Engineer who can write code to achieve the design described by the engineering lead",
        goal="""Write a python module that implements the design described by the engineering lead, in order to achieve the requirements.
            The python module must be completely self-contained, and ready so that it can be tested or have a simple UI built for it.
            Here are the requirements: {requirements}
            The module should be named {module_name} and the class should be named {class_name}""",
        backstory="""You're a seasoned python engineer with a knack for writing clean, efficient code.
            You follow the design instructions carefully.
            You produce 1 python module named {module_name} that implements the design and achieves the requirements.""",
        verbose=True,
        llm=LLM(model="ollama/llama3.2:1b", base_url="http://localhost:11434"),
        allow_code_execution=True,
        code_execution_mode="safe",
        max_execution_time=300,
        max_retry_limit=5,
    )

frontend_engineer = Agent(
        role="A Gradio expert to who can write a simple frontend to demonstrate a backend",
        goal="""Write a gradio UI that demonstrates the given backend, all in one file to be in the same directory as the backend module {module_name}.
            Here are the requirements: {requirements}""",
        backstory="""You're a seasoned python engineer highly skilled at writing simple Gradio UIs for a backend class.
            You produce a simple gradio UI that demonstrates the given backend class; you write the gradio UI in a module app.py that is in the same directory as the backend module {module_name}.""",
        verbose=True,
        llm=LLM(model="ollama/llama3.2:1b", base_url="http://localhost:11434"),
    )

test_engineer = Agent(
        role="An engineer with python coding skills who can write unit tests for the given backend module {module_name}",
        goal="""Write unit tests for the given backend module {module_name} and create a test_{module_name} in the same directory as the backend module.""",
        backstory="""You're a seasoned QA engineer and software developer who writes great unit tests for python code.""",
        verbose=True,
        llm=LLM(model="ollama/llama3.2:1b", base_url="http://localhost:11434"),
        allow_code_execution=True,
        code_execution_mode="safe",
        max_execution_time=300,
        max_retry_limit=5,
    )

### Create Task

In [4]:
design_task = Task(
        description="""Take the high level requirements described here and prepare a detailed design for the engineer;
            everything should be in 1 python module, but outline the classes and methods in the module.
            Here are the requirements: {requirements}
            IMPORTANT: Only output the design in markdown format, laying out in detail the classes and functions in the module, describing the functionality.
            DO NOT WRITE THE PYTHON CODE.""",
        expected_output="A detailed design for the engineer, identifying the classes and functions in the module.",
        agent=engineering_lead,
        output_file="output/{module_name}_design.md"
    )

backend_task = Task(
        description="""Write a python module that implements the design described by the engineering lead, in order to achieve the requirements.
            Here are the requirements: {requirements}""",
        expected_output="""A python module that implements the design and achieves the requirements.
            IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
            The output should be valid Python code that can be directly saved to a file and executed.""",
        agent=backend_engineer,
        context=[design_task],
        output_file="output/{module_name}"
    )

frontend_task = Task(
        description="""Write a gradio UI in a module app.py that demonstrates the given backend class in {module_name}.
            Assume there is only 1 user, and keep the UI very simple indeed - just a prototype or demo.
            Here are the requirements: {requirements}""",
        expected_output="""A gradio UI in module app.py that demonstrates the given backend class.
            The file should be ready so that it can be run as-is, in the same directory as the backend module, and it should import the backend class from {module_name}.
            IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
            The output should be valid Python code that can be directly saved to a file and executed.""",
        agent=frontend_engineer,
        context=[backend_task],
        output_file="output/eng-team-gradio-app.py"
    )

test_task = Task(
        description="""Write unit tests for the given backend module {module_name} and create a test_{module_name} in the same directory as the backend module.""",
        expected_output="""A test_{module_name} module that tests the given backend module.
            IMPORTANT: Output ONLY the raw Python code without any markdown formatting, code block delimiters, or backticks.
            The output should be valid Python code that can be directly saved to a file and executed.""",
        agent=test_engineer,
        context=[backend_task],
        output_file="output/test_{module_name}"
    )

### Create Crew 

In [5]:
crew =  Crew(
        agents=[engineering_lead, backend_engineer, frontend_engineer, test_engineer],
        tasks=[design_task, backend_task, frontend_task, test_task],
        process=Process.sequential,
        verbose=True,
    )

### Run Crew

In [6]:
from IPython.display import Markdown
inputs = {
        'requirements': """
A simple account management system for a trading simulation platform.
The system should allow users to create an account, deposit funds, and withdraw funds.
The system should allow users to record that they have bought or sold shares, providing a quantity.
The system should calculate the total value of the user's portfolio, and the profit or loss from the initial deposit.
The system should be able to report the holdings of the user at any point in time.
The system should be able to report the profit or loss of the user at any point in time.
The system should be able to list the transactions that the user has made over time.
The system should prevent the user from withdrawing funds that would leave them with a negative balance, or
 from buying more shares than they can afford, or selling shares that they don't have.
 The system has access to a function get_share_price(symbol) which returns the current price of a share, and includes a test implementation that returns fixed prices for AAPL, TSLA, GOOGL.
""",
        'module_name': "eng-team-accounts.py",
        'class_name': "Account"
    }

result = crew.kickoff(inputs=inputs)
Markdown(result.raw)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c39fac4a-a7bc-4ff6-9389-a670182b1d18                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 4e3002bf-9cbf-448a-afd0-2d5bb9ee02d7
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 4e3002bf-9cbf-448a-afd0-2d5bb9ee02d7
       Status: Executing Task...
    └── 🤖 Agent: Engineering Lead for the engineering team, directing the work of the engineer
            Status: In Progress

# Agent: Engineering Lead for the engineering team, directing the work of the engineer
## Task: Take the high level requirements described here and prepare a detailed design for the engineer;
            everything should be in 1 python module, but outline the classes and methods in the module.
            Here are the requirements: 
A simple account management system for a trading simulation platform.
The system should allow users to create an account, deposit funds, and withdraw funds.
The system should allow users to record that they have bought or sold shares, providing a quantity.
The system should calculate the total value of the user's portfolio, and the profit or loss from the initial deposit.
The system should be able to report the holdings of the user at any point in time.
The system should be able to report the profit or loss of the user at any point in time.
The system should be able to list the transactions that the user has made over time.
The system should prevent the us

🤖 Agent: Engineering Lead for the engineering team, directing the work of the engineer
    Status: In Progress
└── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 4e3002bf-9cbf-448a-afd0-2d5bb9ee02d7
│      Assigned to: Engineering Lead for the engineering team, directing the work of the engineer
│      Status: ✅ Completed
│   └── 🤖 Agent: Engineering Lead for the engineering team, directing the work of the engineer
│           Status: ✅ Completed
├── 📋 Task: 20e10a96-5430-4425-a1da-a26dd784b99e
│      Assigned to: Python Engineer who can write code to achieve the design described by the engineering lead
│      Status: ✅ Completed
│   └── 🤖 Agent: Python Engineer who can write code to achieve the design described by the engineering lead
│           Status: ✅ Completed
├── 📋 Task: 95194f2e-e39b-4766-bc0a-af72dd9677ad
│      Assigned to: A Gradio expert to who can write a simple frontend to demonstrate a backend
│      Status: ✅ Completed
│   └── 🤖 Agent: A Gradio expert to who can write a simple frontend to demonstrate a backend
│           Status: ✅ Completed
└── 📋 Task: 040bd50e-15aa-4723-bc3b-b0727ae03958
       Status: Executing Task...
    └── 🤖 Agent: An engineer with python coding skills who can write unit tests for the given backend module 
        eng-team-accounts.py
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 4e3002bf-9cbf-448a-afd0-2d5bb9ee02d7
│      Assigned to: Engineering Lead for the engineering team, directing the work of the engineer
│      Status: ✅ Completed
│   └── 🤖 Agent: Engineering Lead for the engineering team, directing the work of the engineer
│           Status: ✅ Completed
├── 📋 Task: 20e10a96-5430-4425-a1da-a26dd784b99e
│      Assigned to: Python Engineer who can write code to achieve the design described by the engineering lead
│      Status: ✅ Completed
│   └── 🤖 Agent: Python Engineer who can write code to achieve the design described by the engineering lead
│           Status: ✅ Completed
├── 📋 Task: 95194f2e-e39b-4766-bc0a-af72dd9677ad
│      Assigned to: A Gradio expert to who can write a simple frontend to demonstrate a backend
│      Status: ✅ Completed
│   └── 🤖 Agent: A Gradio expert to who can write a simple frontend to demonstrate a backend
│           Status: ✅ Completed
└── 📋 Task: 040bd50e-15aa-4723-bc3b-b0727ae03958
       Assigned to: An engineer with python coding skills who can write unit tests for the given backend module 
    eng-team-accounts.py
       Status: ✅ Completed
    └── 🤖 Agent: An engineer with python coding skills who can write unit tests for the given backend module 
        eng-team-accounts.py
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 040bd50e-15aa-4723-bc3b-b0727ae03958                                                                     │
│  Agent: An engineer with python coding skills who can write unit tests for the given backend module             │
│  eng-team-accounts.py                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c39fac4a-a7bc-4ff6-9389-a670182b1d18                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

```python
import unittest
from unittest.mock import patch, MagicMock
from eng_team_accounts import get_share_price, Account

class TestEngTeamAccounts(unittest.TestCase):

    @patch('eng_team_accounts.Code Interpreter.input')
    def test_get_share_price(self, mock_input):
        mock_input.return_value = '10.5'
        result = get_share_price('AAPL')
        self.assertAlmostEqual(result, 10.5)

    @patch('eng_team_accounts.Code Interpreter.output')
    def test_deposit_account(self, mock_output):
        account = Account("John Doe", 100)
        mock_output.side_effect = [f"Deposited ${amount} into account {account.id}", f"Withdrew ${amount} from account {account.id}" for amount in [10, 20]]
        result = account.deposit(50)
        self.assertEqual(result, '{"account": {"id": len(json.dumps({"account": {}})) - 1, "owner_name": "John Doe", "initial_balance": 150, "portfolio": {}, "transactions": []}}')

    @patch('eng_team_accounts.Code Interpreter.output')
    def test_withdraw_account(self, mock_output):
        account = Account("John Doe", 100)
        mock_output.side_effect = [f"Withdrew ${amount} from account {account.id}", f"Insufficient balance or cannot afford the transaction for {amount} shares of AAPL" for amount in [10, 20]]
        result = account.withdraw(50)
        self.assertEqual(result, '{"account": {"id": len(json.dumps({"account": {}})) - 1, "owner_name": "John Doe", "initial_balance": 150, "portfolio": {}, "transactions": []}}')

    @patch('eng_team_accounts.Code Interpreter.output')
    def test_record_transaction(self, mock_output):
        account = Account("John Doe", 100)
        result = account.record_transaction('AAPL', 10, 15.0)
        self.assertEqual(result, '{"account": {"id": len(json.dumps({"account": {}})) - 1, "owner_name": "John Doe", "initial_balance": 150, "portfolio": {}, "transactions": [{"symbol": "AAPL", "quantity": 10, "price": 15.0}]}}')

    @patch('eng_team_accounts.Code Interpreter.output')
    def test_get_portfolio(self, mock_output):
        account = Account("John Doe", 100)
        result = account.get_portfolio()
        self.assertEqual(result, '{"account": {"id": len(json.dumps({"account": {}})) - 1, "owner_name": "John Doe", "initial_balance": 150, "portfolio": {}, "transactions": []}}')

    @patch('eng_team_accounts.Code Interpreter.output')
    def test_get_balance(self, mock_output):
        account = Account("John Doe", 100)
        result = account.get_balance()
        self.assertEqual(result, '{"account": {"id": len(json.dumps({"account": {}})) - 1, "owner_name": "John Doe", "initial_balance": 150, "portfolio": {}, "transactions": []}}')

    @patch('eng_team_accounts.Code Interpreter.output')
    def test_list_transactions(self, mock_output):
        account = Account("John Doe", 100)
        result = account.list_transactions()
        self.assertEqual(result, '{"account": {"id": len(json.dumps({"account": {}})) - 1, "owner_name": "John Doe", "initial_balance": 150, "portfolio": {}, "transactions": [{"symbol": "AAPL", "quantity": 10, "price": 15.0}]}}')

if __name__ == '__main__':
    unittest.main()
```